# 6.1 텍스트 데이터 다루기

### 텍스트 데이터

- **텍스트는 단어의 시퀀스나 문자의 시퀀스**로 이해할 수 있으며 가장 흔한 시퀀스 형태의 
데이터임


- 텍스트 데이터를 신경망에 주입시키기 위해서는 다음의 2가지 단계를 거쳐야 함
    1. 토큰화
    2. 벡터화


### 토큰화와 벡터화

#### 토큰화(tokenization)

- 텍스트를 나누는 단위(단어, 문자, n-그램)를 **토큰(token)**이라 함
- **텍스트를 토큰으로 나누는 작업을 토큰화(tokenization)**라고 함

#### 벡터화(vectorization)

- **벡터화(vectorization)는 토큰화를 거친 토큰들을 벡터로 변환해 주는 작업**을 말함

- 여기서 말하는 벡터화는 텍스트 데이터를 다루는 경우를 말하며, 일반적인 의미의 벡터화는 데이터를 수치형 텐서로 바꿔주는 작업을 뜻함

#### 텍스트 벡터화의 여러가지 방식들
- 텍스트를 **단어**로 나누고 각 단어를 하나의 벡터로
- 텍스트를 **문자**로 나누고 각 문자를 하나의 벡터로
- 텍스트에서 단어나 문자의 **n-그램(n-gram)**을 추출해 각 n-그램을 하나의 벡터로

위의 과정 모두 토큰화를 거쳐 벡터화가 되는 것을 알 수 있음

#### 참고) n-그램과 BoW

- **n-그램은 문장에서 추출한 N개의 연속된 단어 그룹**을 말함


- 이러한 n-그램 집합은 리스트나 시퀀스가 아니라 단순히 **토큰의 집합**이며 순서가 없다는 특징이 있으며 이러한 종류의 토큰화 방법을 **Bow(Bag-of-Words)**라고 함


- **BoW는 순서가 없는 토큰화 방법이기 때문에 n-그램을 추출하는 것은 일종의 특성 공학**임
    
- 따라서 머신러닝(얕은학습)의 텍스트 처리 모델에서는 유용한 특성공학 방법이지만 **딥러닝에서는 순환신경망과 1D컨브넷으로 이러한 특성 공학을 대체**함


### 토큰과 벡터의 연결

여기서는 토큰과 벡터를 연결하는 방식 중 주요한 2가지 방법을 소개함

**1. 원-핫 인코딩(one-hot encoding)**  

**2. 토큰 임베딩(token embedding)**

## 6.1.1 단어와 문자의 원-핫 인코딩

### 원-핫 인코딩

- 원-핫 인코딩(one-hot encoding)은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법

- 모든 단어에 고유한 정수 인덱스 i를 부여하고 이 정수 인덱스i를 크기가 N(단어 사전의 크기)인 이진 벡터로 변환

### 단어, 문자 수준의 one-hot encoding 구현 예

In [1]:
# 단어 수준의 one-hot encoding의 간단한 그현 예

import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework']

token_index = {}

# 단어 사전 생성 (index 0은 사용 X)
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index)+1

# 한 문장에 총 10개의 단어만 제한
max_length = 10

result = np.zeros((len(samples), max_length, max(token_index.values())+1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
#         index = token_index.get(word)
#         result[i, j, index] = 1.
        result[i, j, token_index[word]] = 1.

In [2]:
# 문자 수준 one-hot encoding의 간단한 구현 예

import string

samples = ['The cat sat on the mat.', 'The dog ate my homework']

# 출력할 수 있는 모든 아스키 문자들로 된 문자사전 생성
characters = string.printable
token_index = dict(zip(characters, range(1, len(characters)+1)))

# 문장당 최대 문자 50자
max_length = 50

result = np.zeros((len(samples), max_length, max(token_index.values())+1))

for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
#         index = token_index.get(character)
#         result[i, j, index] = 1.
        result[i, j, token_index[character]] = 1.

### 케라스 유틸리티 사용 one-hot encoding

#### `Tokenizer` 클래스

- 케라스의 `Tokenizer`클래스를 사용하면 단어 수준의 one-hot encoding을 할 수 있음


- 객체 생성 시 `num_words` 매개변수로 단어 수 제한(가장 높은 빈도의 단어만 제한된 수 내에서 선택)


- `texts_to_matrix()`
    - `texts_to_matrix()`는 아래의 두 메서드를 차례대로 호출
    - 텍스트를 시퀀스 리스트로 바꾸어주는 `texts_to_sequences()`
    - 시퀀스 리스트를 넘파이 배열로 바꾸어주는 `sequences_to_matrix()`
    

- `texts_to_matrix()`의 `mode` 매개변수는 binary 말고도 count, freq, tfidf 등이 있음
    - binary 지정시 이진 벡터 표현을 얻을 수 있음

In [3]:
# 케라스를 사용한 단어 수준의 one-hot encoding

from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework']

# 가장 빈도가 높은 1000개의 단어만 선택하도록 Tokenizer 객체를 생성
tokenizer = Tokenizer(num_words=1000)

# 샘플 데이터를 바탕으로 단어 인덱스를 생성
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index
print(word_index)

Using TensorFlow backend.


{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


### 원-핫 해싱(one-hot hashing)

- 원-핫 인코딩의 변종으로 어휘사전의 토큰 수가 너무 커서 다루기 어려울 때 사용
- 단어를 해싱해 고정된 크기의 벡터로 변환하는 방식


- 메모리를 아낄 수 있으며 온라인 방식으로 데이터를 인코딩 할 수 있는 장점이 있음
- 해시 충돌(hash collision)이 일어날 수 있다는 단점이 있음(다른 단어가 같은 해시를 만드는 경우)

In [4]:
# 단어 수준 one-hot hashing의 간단한 구현 예

samples = ['The cat sat on the mat.', 'The dog ate my homework']

# 단어를 크기가 1000인 벡터로 저장
# 1000개 이상의 단어가 있으며 해시 충돌 일어날 수 있음
dimensionality = 1000

max_length = 10

result = np.zeros((len(samples), max_length, dimensionality))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality
        result[i, j, index] = 1.